## IMPORT

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 3.4 MB/s 
     |████████████████████████████████| 462 kB 16.7 MB/s 
     |████████████████████████████████| 4.2 MB 7.8 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [3]:
# cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v2.1"

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


In [6]:
print(os.getcwd())

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


## Preparing the dataset

In [7]:
import userEmbedding
builder = tfds.builder('Userembedding')
userEmbeddingDs = tfds.load('Userembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userembedding/1.0.0.incompleteZCO0N3/userembedding-train.tfrecord*...:   0…

Dataset userembedding downloaded and prepared to /root/tensorflow_datasets/userembedding/1.0.0. Subsequent calls will reuse this data.


In [8]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incomplete7W3F50/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [9]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incompleteWNP55H/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [10]:
import wishFeatures
builder = tfds.builder('Wishfeatures')
wishFeaturesDs = tfds.load('Wishfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishfeatures/1.0.0.incompleteQ6ESMI/wishfeatures-train.tfrecord*...:   0%|…

Dataset wishfeatures downloaded and prepared to /root/tensorflow_datasets/wishfeatures/1.0.0. Subsequent calls will reuse this data.


In [29]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x["like"]
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

In [30]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0, 'like': 1, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'like': 1, 'location_name': b'DUBAI', 'user_id': b'112'}


In [28]:
for x in userFeatureDs.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0,
 'category': b'culinary',
 'like': 1,
 'location_id': b'61',
 'location_name': b'JAKARTA',
 'user_id': b'93'}
{'add': 0,
 'category': b'beach',
 'like': 1,
 'location_id': b'53',
 'location_name': b'DUBAI',
 'user_id': b'112'}


In [31]:
NUM_DATA = ratings.__len__().numpy()

tf.random.set_seed(42)
shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [32]:
class RetrievalModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32
    

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [49]:
model = RetrievalModel(rating_weight=2.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
# model.build(input_shape=[])
# model.summary()

In [50]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [51]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
43/43 [==============================] - 5s 73ms/step - root_mean_squared_error: 0.4516 - factorized_top_k/top_1_categorical_accuracy: 0.0023 - factorized_top_k/top_5_categorical_accuracy: 0.0340 - factorized_top_k/top_10_categorical_accuracy: 0.0748 - factorized_top_k/top_50_categorical_accuracy: 0.4006 - factorized_top_k/top_100_categorical_accuracy: 0.8192 - loss: 0.4078 - regularization_loss: 0.0000e+00 - total_loss: 0.4078
Epoch 2/10
43/43 [==============================] - 3s 70ms/step - root_mean_squared_error: 0.4457 - factorized_top_k/top_1_categorical_accuracy: 0.0048 - factorized_top_k/top_5_categorical_accuracy: 0.0387 - factorized_top_k/top_10_categorical_accuracy: 0.0799 - factorized_top_k/top_50_categorical_accuracy: 0.4141 - factorized_top_k/top_100_categorical_accuracy: 0.8262 - loss: 0.3974 - regularization_loss: 0.0000e+00 - total_loss: 0.3974
Epoch 3/10
43/43 [==============================] - 3s 69ms/step - root_mean_squared_error: 0.4449 - factorized_to

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [44]:
model = RetrievalModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [45]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
43/43 [==============================] - 4s 71ms/step - root_mean_squared_error: 0.5028 - factorized_top_k/top_1_categorical_accuracy: 0.0938 - factorized_top_k/top_5_categorical_accuracy: 0.3316 - factorized_top_k/top_10_categorical_accuracy: 0.4394 - factorized_top_k/top_50_categorical_accuracy: 0.7208 - factorized_top_k/top_100_categorical_accuracy: 0.9139 - loss: 3175.9027 - regularization_loss: 0.0000e+00 - total_loss: 3175.9027
Epoch 2/10
43/43 [==============================] - 3s 68ms/step - root_mean_squared_error: 0.5022 - factorized_top_k/top_1_categorical_accuracy: 0.5215 - factorized_top_k/top_5_categorical_accuracy: 0.9153 - factorized_top_k/top_10_categorical_accuracy: 0.9567 - factorized_top_k/top_50_categorical_accuracy: 0.9919 - factorized_top_k/top_100_categorical_accuracy: 0.9990 - loss: 3175.7500 - regularization_loss: 0.0000e+00 - total_loss: 3175.7500
Epoch 3/10
43/43 [==============================] - 3s 69ms/step - root_mean_squared_error: 0.5028 - f

In [46]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 1s 66ms/step - root_mean_squared_error: 0.5094 - factorized_top_k/top_1_categorical_accuracy: 0.0089 - factorized_top_k/top_5_categorical_accuracy: 0.0639 - factorized_top_k/top_10_categorical_accuracy: 0.6226 - factorized_top_k/top_50_categorical_accuracy: 0.8077 - factorized_top_k/top_100_categorical_accuracy: 0.9672 - loss: 1344.8006 - regularization_loss: 0.0000e+00 - total_loss: 1344.8006


{'factorized_top_k/top_100_categorical_accuracy': 0.9672130942344666,
 'factorized_top_k/top_10_categorical_accuracy': 0.6225865483283997,
 'factorized_top_k/top_1_categorical_accuracy': 0.008925318717956543,
 'factorized_top_k/top_50_categorical_accuracy': 0.8076502680778503,
 'factorized_top_k/top_5_categorical_accuracy': 0.06393442302942276,
 'loss': 539.7352905273438,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.509373128414154,
 'total_loss': 539.7352905273438}

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [47]:
model = RetrievalModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [48]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 4s 70ms/step - root_mean_squared_error: 0.4536 - factorized_top_k/top_1_categorical_accuracy: 0.1214 - factorized_top_k/top_5_categorical_accuracy: 0.2203 - factorized_top_k/top_10_categorical_accuracy: 0.2861 - factorized_top_k/top_50_categorical_accuracy: 0.6130 - factorized_top_k/top_100_categorical_accuracy: 0.8704 - loss: 3176.0203 - regularization_loss: 0.0000e+00 - total_loss: 3176.0203
Epoch 2/3
43/43 [==============================] - 3s 69ms/step - root_mean_squared_error: 0.4464 - factorized_top_k/top_1_categorical_accuracy: 0.4883 - factorized_top_k/top_5_categorical_accuracy: 0.7717 - factorized_top_k/top_10_categorical_accuracy: 0.8549 - factorized_top_k/top_50_categorical_accuracy: 0.9814 - factorized_top_k/top_100_categorical_accuracy: 0.9993 - loss: 3175.9342 - regularization_loss: 0.0000e+00 - total_loss: 3175.9342
Epoch 3/3
22/22 [==============================] - 2s 63ms/step - root_mean_squared_error: 0.4469 - fact